#### Delete the file with the reserved name, if it exists, and create a file to merge the contents

In [93]:
xlsx = 'unite.xlsx'

from glob import glob
def capacity(): return glob('*.xlsx') 
def check(message): print(message, '\n', capacity(), '\n')

check('Before launch it was like this :')

from os import remove
from os.path import isfile
if isfile(xlsx): 
    remove(xlsx)
    check('Deleted the file named unite.xlsx :')
fxls = sorted(capacity())

from pandas import ExcelWriter
ExcelWriter(xlsx).save()
check('Created a file named unite.xlsx :')

Before launch it was like this : 
 ['Book1.xlsx', 'unite.xlsx', 'Book3.xlsx', 'Book2.xlsx'] 

Deleted the file named unite.xlsx : 
 ['Book1.xlsx', 'Book3.xlsx', 'Book2.xlsx'] 

Created a file named unite.xlsx : 
 ['Book1.xlsx', 'unite.xlsx', 'Book3.xlsx', 'Book2.xlsx'] 



### Looping through files and pages of files to save their contents to a new file

In [94]:
from pandas import ExcelFile, read_excel

def supplementation(this, file, sheet):
    writer = ExcelWriter(file, mode="a", engine="openpyxl")
    this.to_excel(writer, sheet_name=f"{sheet}", index=False)
    writer.save()

def unite():
    list_sheet=[]

    for fxl in fxls:
        ef = ExcelFile(fxl)
        print('\n', fxl, ":", end=' ')
        for sheet in ef.sheet_names:
            list_sheet.append(sheet) if sheet not in list_sheet else list_sheet.append(sheet+'_'+fxl.replace('.xlsx',''))
            supplementation(this=read_excel(ef, sheet_name=sheet), file=xlsx, sheet=list_sheet[-1])
            print(list_sheet[-1], end=' ')

unite()


 Book1.xlsx : FIRST 
 Book2.xlsx : FIRST_Book2 SECOND 
 Book3.xlsx : FIRST_Book3 SECOND_Book3 THIRD 

#### When we create a file, a page with a known name is also created, we can rename this page

In [95]:
from openpyxl import load_workbook

def rename(file, sheet):
    fxl = load_workbook(file)
    fxl['Sheet1'].title = sheet
    fxl.save(file)

def verify(file, message='Result:'):
    fxl = load_workbook(file)
    sheets = fxl.sheetnames
    print(message, sheets)

verify(xlsx, 'Before:')
chief = 'chief'
rename(xlsx, chief)
verify(xlsx, 'After:')

Before: ['Sheet1', 'FIRST', 'FIRST_Book2', 'SECOND', 'FIRST_Book3', 'SECOND_Book3', 'THIRD']
After: ['chief', 'FIRST', 'FIRST_Book2', 'SECOND', 'FIRST_Book3', 'SECOND_Book3', 'THIRD']


### Add data to empty page

In [96]:
cell = 'A1'
content = 'HOWDY GUEST'

def fill(file, namesheet, cell, content):
    fxl = load_workbook(file)
    sheet = fxl[namesheet]
    sheet[cell]= str(content)
    fxl.save(file)
    print(f'The "{content}" content has been added to cell "{cell}" of the "{namesheet}" page of the "{file}" file.')

fill(file=xlsx, namesheet=chief, cell=cell, content=content)

The "HOWDY GUEST" content has been added to cell "A1" of the "chief" page of the "unite.xlsx" file.


### Read information from a cell

In [97]:
def scan(file, namesheet, cell):
    fxl = load_workbook(f'{file}',read_only=False, keep_vba= True)
    cell_obj = fxl[namesheet][cell]
    return cell_obj.value

output = scan(file=xlsx, namesheet=chief, cell=cell)
print(f'In cell "{cell}" of the "{chief}" page of the file "{xlsx}" we got the contents of "{output}".')

In cell "A1" of the "chief" page of the file "unite.xlsx" we got the contents of "HOWDY GUEST".


### To see the result in the file run this...

In [98]:
from os import system
system('open unite.xlsx')

0